In [1]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded

if not is_dbenv_loaded():
    load_dbenv()

In [2]:
import sys
import argparse
import pymatgen
import numpy as np
import matplotlib.pyplot as plt
from aiida.work.run import run
from aiida.orm.data.upf import UpfData
from aiida.common.exceptions import NotExistent
from aiida.orm.data.upf import get_pseudos_from_structure
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.structure import StructureData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.array import ArrayData
from aiida.orm.data.folder import FolderData
from aiida.orm.data.remote import RemoteData
from aiida.orm import DataFactory
from aiida.orm.data.singlefile import SinglefileData

from aiida.orm.code import Code
from aiida.orm import load_node

from aiida.work.run import run, submit
from aiida.work.workfunction import workfunction
from aiida.work.workchain import WorkChain, ToContext, while_, Outputs, if_, append_
from aiida.orm.data.base import Float, Str, NumericType, BaseType, Int, Bool, List
from aiida_defects.formation_energy.pot_align import  PotentialAlignmentLanyZunger

In [3]:
codename="pw_5.1@ubelix"
code_pp="pp_5.1@ubelix"
pseudo_family='SSSP'



options={
        'resources': {
            'num_machines': 2,
            #'num_mpiprocs_per_machine': 1,
        },
         'max_wallclock_seconds' : 36000,
         #'custom_scheduler_commands' : u"#SBATCH --partition=all",
         #'custom_scheduler_commands' : u"#SBATCH --account=dcb",
         'custom_scheduler_commands' : u"#SBATCH --partition=empi",

        }

settings={}

kpoints = KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])
kpoints = KpointsData()
kpoints.set_kpoints_mesh([3, 3, 3])


parameters = {
        'CONTROL': {
            'restart_mode': 'from_scratch',
            #'tstress': True,
            #'tprnfor' : True,
            'nstep' : 500,
            'etot_conv_thr' : 1.0e-6,
             'forc_conv_thr' : 1.0e-3,
            #'tefield'      : True,
            #'dipfield'  : True
        },
        'SYSTEM': {
            'ecutwfc': 40.,
            'ecutrho': 320.,
            'occupations' : 'smearing',
            'degauss' : 0.01,
#             'edir'   : 3,
#             'emaxpos' : 0.75,
#             'eopreg': 0.01,
#             'eamp' : 0.0,

            #'starting_magnetization' : starting_magnetization,
        },
        'ELECTRONS': {
            'conv_thr': 1.e-8,
            'mixing_beta' : 0.6,
            'startingwfc' : 'atomic',
        },

    }

parameters_def = {
        'CONTROL': {
            'restart_mode': 'from_scratch',
            #'tstress': True,
            #'tprnfor' : True,
            'nstep' : 500,
            'etot_conv_thr' : 1.0e-6,
             'forc_conv_thr' : 1.0e-3,
            #'tefield'      : True,
            #'dipfield'  : True
        },
        'SYSTEM': {
            'ecutwfc': 40.,
            'ecutrho': 320.,
            'occupations' : 'smearing',
            'degauss' : 0.01,
#             'edir'   : 3,
#             'emaxpos' : 0.75,
#             'eopreg': 0.01,
#             'eamp' : 0.0,
            'tot_charge' : 2,

            #'starting_magnetization' : starting_magnetization,
        },
        'ELECTRONS': {
            'conv_thr': 1.e-8,
            'mixing_beta' : 0.6,
            'startingwfc' : 'atomic',
        },

    }

parameters_pp = {
        'INPUTPP': {'plot_num' : 11,
        }
        }


#Storing nd structure
inputfile='/home/ricca/Desktop/PostDoc_2017-2018/ZrO2_ETH/cristiana/nd.cif'


from pymatgen.io import cif,aiida
from pymatgen.io.aiida import AiidaStructureAdaptor

structure_mg= cif.Structure.from_file(str(inputfile))
aiida_structure_adaptor = AiidaStructureAdaptor()
host= aiida_structure_adaptor.get_structuredata(structure_mg)


inputfile='/home/ricca/Desktop/PostDoc_2017-2018/ZrO2_ETH/cristiana/f2+.cif'
structure_mg= cif.Structure.from_file(str(inputfile))
aiida_structure_adaptor = AiidaStructureAdaptor()
f2= aiida_structure_adaptor.get_structuredata(structure_mg)

outputs=run(PotentialAlignmentLanyZunger,
            host_structure=host,
            defect_structure=f2,
            code_pp=Str(code_pp),
            code_pw=Str(codename),
            run_pw_defect=Bool(True),
            run_pw_host=Bool(True),
            settings=ParameterData(dict=settings),
            options=ParameterData(dict=options),
            pseudo_family=Str(pseudo_family),
            kpoints=kpoints,
            parameters_pp=ParameterData(dict=parameters_pp),
            parameters_pw_host=ParameterData(dict=parameters),
            parameters_pw_defect=ParameterData(dict=parameters_def)
           )


12/19/2018 08:55:03 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_host]: Launching PpWorkChain for the host. pk value 166417
12/19/2018 09:02:54 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_host]: Launching PpWorkChain for atom kind name 'Zr' @ 7.23114392,7.22703072,10.4560705996 in the host system. pk value 166587
12/19/2018 09:02:54 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_host]: Launching PpWorkChain for atom kind name 'Zr' @ 1.76466472,1.76617168,2.54891740167 in the host system. pk value 166589
12/19/2018 09:02:55 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_host]: Launching PpWorkChain for atom kind na

12/19/2018 09:03:02 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_host]: Launching PpWorkChain for atom kind name 'O' @ 7.23883888,5.40353632,6.19009396704 in the host system. pk value 166643
12/19/2018 09:03:03 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_host]: Launching PpWorkChain for atom kind name 'O' @ 1.76855952,7.21144424,9.44819459586 in the host system. pk value 166645
12/19/2018 09:03:03 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_host]: Launching PpWorkChain for atom kind name 'O' @ 3.58679776,5.38867056,8.83292267961 in the host system. pk value 166647
12/19/2018 09:03:03 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlig

12/19/2018 09:16:19 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_defect]: Launching PpWorkChain for atom kind name 'Zr' @ 3.5731332,7.22057336,10.42406065 in defective system. pk value 167057
12/19/2018 09:16:19 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_defect]: Launching PpWorkChain for atom kind name 'Zr' @ 5.40593144,1.79045048,2.55944685168 in defective system. pk value 167059
12/19/2018 09:16:20 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_defect]: Launching PpWorkChain for atom kind name 'Zr' @ 3.55397952,7.2282028,5.21477599239 in defective system. pk value 167061
12/19/2018 09:16:20 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|Poten

12/19/2018 09:16:27 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_defect]: Launching PpWorkChain for atom kind name 'O' @ 5.407038,3.57327152,9.42871669434 in defective system. pk value 167113
12/19/2018 09:16:27 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_defect]: Launching PpWorkChain for atom kind name 'O' @ 7.24190376,1.75816368,8.8522214013 in defective system. pk value 167115
12/19/2018 09:16:28 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|PotentialAlignmentLanyZunger|run_atomic_sphere_pot_defect]: Launching PpWorkChain for atom kind name 'O' @ 5.40989904,3.58512336,6.73520116986 in defective system. pk value 167117
12/19/2018 09:16:28 AM, aiida.aiida.orm.implementation.general.calculation.work.WorkCalculation: [REPORT] [166415|Potenti